In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
import seaborn as sns
pd.set_option('display.max_columns',None)
from sklearn.metrics import roc_auc_score
import pickle
TARGET = 'answered_correctly'

In [2]:
with open('../models/069_simple_lgb.pkl','rb') as f:
    model1 = pickle.load(f)

In [8]:
with open('../models/070_simple_lgb.pkl','rb') as f:
    model2 = pickle.load(f)

In [9]:
with open('../models/071_simple_lgb_big.pkl','rb') as f:
    model3 = pickle.load(f)

In [10]:
USE_COLS = ['prior_question_elapsed_time', 'prior_question_had_explanation', 'part',
            'ans_user_avg', 'ans_user_count','elapsed_time_user_avg', 'explanation_user_avg','elapsed_time_user_sum',
            'ans_content_avg', 'elapsed_time_content_avg', 'explanation_content_avg',
            'user_content_count',
            'lag_time_1', 'lag_time_2', 'lag_time_3','lag_incorrect_time',
            'lag_part_time_1','lag_part_time_2','lag_part_time_3',
            "tags_pca_0", "tags_pca_1",
            'first_bundle',
            'ans_part_mean','ans_user_part_avg','user_part_count',
            'ans_community_mean','community',
            'content_id','bundle_id',
            'first_bundle_cut','first_bundle_cut_mean',
            'rolling_mean_10','rolling_mean_3']

In [11]:
def load_features(features_list,path,train_valid):

    dfs = [pd.read_feather(f'../{path}/{feature}_{train_valid}.feather') for feature in features_list]
    df = pd.concat(dfs,axis=1)
    return df

In [12]:
path = 'all_data'
FEATURES_LIST = ['BASE_FIX','LOOP_FIX3','TAGS','GROUP_BY','BUNDLE_ID','ROLLING_MEAN']
valid = load_features(FEATURES_LIST,path=f'features/{path}',train_valid='valid')

In [13]:
valid['pred1'] = model1.predict(valid[USE_COLS])
valid['pred2'] = model2.predict(valid[USE_COLS])

In [17]:
valid['pred3'] = model3.predict(valid[USE_COLS])

In [14]:
roc_auc_score(valid[TARGET],valid['pred1'])

0.7848071122915671

In [15]:
roc_auc_score(valid[TARGET],valid['pred2'])

0.7846245795206664

In [18]:
roc_auc_score(valid[TARGET],valid['pred3'])

0.7890502973360569

In [21]:
roc_auc_score(valid[TARGET],valid['pred1']+valid['pred3']*10)

0.7889486913660605

In [2]:
val = pd.read_feather('../features/kernel_base/BASE_FIX_valid.feather')
oof1 = pd.read_feather('../data/oof/064_simple_lgb.feather')
oof2 = pd.read_feather('../data/oof/065_simple_lgb.feather')
# oof3 = pd.read_feather('../data/oof/060_simple_lgb.feather')

In [3]:
oof1.columns = ['row_id','pred1']
oof2.columns = ['row_id','pred2']
# oof3.columns = ['row_id','pred3']

In [4]:
df = pd.concat([val,oof1['pred1'],oof2['pred2']],axis=1)

In [14]:
df['pred'] = df['pred1'] + df['pred2']*1.5

In [15]:
roc_auc_score(df[TARGET],df['pred'])

0.7843040247864422